In [1]:
!pip install diffusers transformers accelerate gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127

In [2]:
import torch
import gradio as gr
from PIL import Image
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline

# Check GPU
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load models (optimized to load only once)
text_to_img_pipe = StableDiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-1",
    torch_dtype=torch.float16
).to(device)

img2img_pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
).to(device)

animegan_model = torch.hub.load(
    "bryandlee/animegan2-pytorch:main",
    "generator",
    pretrained=True,
    device=device
)
face2paint = torch.hub.load(
    "bryandlee/animegan2-pytorch:main",
    "face2paint",
    size=512,
    device=device
)

# Combined function: Text → Anime → Realistic
def generate_realistic_interior(prompt, strength=0.4):
    # Step 1: Generate anime image from text
    anime_image = text_to_img_pipe(prompt).images[0]
    anime_image = anime_image.convert("RGB")

    # Step 2: Apply AnimeGANv2 (optional, comment if unwanted)
    anime_image = face2paint(animegan_model, anime_image)

    # Step 3: Convert anime to realistic
    realistic_image = img2img_pipe(
        prompt="high-quality realistic interior design, photorealistic, 4k",
        image=anime_image,
        strength=strength
    ).images[0]

    return realistic_image

# Gradio UI (simplified)
with gr.Blocks() as demo:
    gr.Markdown("# 🏡 Auto-Realistic Interior Generator")

    with gr.Row():
        prompt_input = gr.Textbox(label="Describe your interior", placeholder="e.g., A minimalist Scandinavian living room")
        strength_slider = gr.Slider(0.1, 0.8, value=0.5, label="Realism Strength")
        generate_btn = gr.Button("Generate Realistic Design")

    output_image = gr.Image(label="Realistic Interior Output")

    generate_btn.click(
        fn=generate_realistic_interior,
        inputs=[prompt_input, strength_slider],
        outputs=output_image
    )

demo.launch()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/537 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/824 [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/bryandlee/animegan2-pytorch/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://github.com/bryandlee/animegan2-pytorch/raw/main/weights/face_paint_512_v2.pt" to /root/.cache/torch/hub/checkpoints/face_paint_512_v2.pt
100%|██████████| 8.20M/8.20M [00:00<00:00, 147MB/s]
Using cache found in /root/.cache

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ed77307a8873244992.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [4]:
!pip install gradio_client

In [7]:
import torch
import gradio as gr
from PIL import Image
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline
from gradio_client import Client

# Check GPU
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load models (optimized to load only once)
text_to_img_pipe = StableDiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-1",
    torch_dtype=torch.float16
).to(device)

img2img_pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
).to(device)

animegan_model = torch.hub.load(
    "bryandlee/animegan2-pytorch:main",
    "generator",
    pretrained=True,
    device=device
)
face2paint = torch.hub.load(
    "bryandlee/animegan2-pytorch:main",
    "face2paint",
    size=512,
    device=device
)

# Combined function: Text → Anime → Realistic
def generate_realistic_interior(prompt, strength=0.4):
    # Step 1: Generate anime image from text
    anime_image = text_to_img_pipe(prompt).images[0]
    anime_image = anime_image.convert("RGB")

    # Step 2: Apply AnimeGANv2 (optional, comment if unwanted)
    anime_image = face2paint(animegan_model, anime_image)

    # Step 3: Convert anime to realistic
    realistic_image = img2img_pipe(
        prompt="high-quality realistic interior design, photorealistic, 4k",
        image=anime_image,
        strength=strength
    ).images[0]

    return realistic_image

# Run Gradio as an API
demo = gr.Interface(
    fn=generate_realistic_interior,
    inputs=["text", gr.Slider(0.1, 0.8, value=0.5, label="Realism Strength")],
    outputs="image"
)

demo.launch(share=True)

# Client Request to the API
client = Client("https://ed77307a8873244992.gradio.live/")
result = client.predict(
    prompt="A cozy modern bedroom with warm lighting",
    strength=0.5,
    api_name="/generate_realistic_interior"
)

# Print result (This should be the generated image)
print(result)


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Using cache found in /root/.cache/torch/hub/bryandlee_animegan2-pytorch_main
Using cache found in /root/.cache/torch/hub/bryandlee_animegan2-pytorch_main


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7af5ba8934fbed7bf2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Loaded as API: https://ed77307a8873244992.gradio.live/ ✔


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

/tmp/gradio/085106b2db69924b0bd6478de92ad29323c0f8c81d2eb509e09ea9f259acb609/image.webp


In [8]:
!pip install flask flask_cors pyngrok openai


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from transformers import pipeline

app = Flask(__name__)
CORS(app)  # Enable CORS for frontend integration

chatbot = pipeline("text-generation", model="microsoft/DialoGPT-medium")

@app.route('/chat', methods=['POST'])
def chat():
    data = request.json
    user_input = data.get("message", "")
    response = chatbot(user_input, max_length=100, do_sample=True)
    return jsonify({"response": response[0]['generated_text']})

if __name__ == '__main__':
    app.run()


config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

...

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

In [ ]:
from pyngrok import ngrok

# Start ngrok tunnel to expose the Flask API
public_url = ngrok.connect(5000)
print(f"Public chatbot URL: {public_url}")
